# Apéndice C. Optimización de algoritmos en procesadores con soporte SIMD

En los Capítulos 3, 5 y 6 se presentaron varias estrategias para la construcción de un sistema de procesamiento digital de señales de manera optimizada, usando diferentes estructuras de implementación y aritmética en punto fijo. A continuación se mostrará una técnica conocida con el nombre de *_loop unrolling_* que permite aprovechar el paralelismo de los microprocesadores modernos, ilustrando un ejemplo de aplicación en la implementación de un filtro FIR.

**¿Qué es _Loop Unrolling_?**.  Muchos microprocesadores modernos incluyen conjuntos de instrucciones multimedia que implementan lo que se denomina instrucciones *SIMD* (_Single Instruction Multiple Data_), por ejemplo, los chips Intel cuentan con el repertorio de instrucciones AVX (_Advanced Vector Extensions_) y ARM lo que se denomina instrucciones NEON. Para entender como estos conjuntos de instrucciones pueden mejorar el desempeño de nuestros algoritmos analicemos un filtro FIR típico implementado en estructura directa:

```C
for(n=0;n<N_DATA;n++){
    y[n] = 0;
    for(k=0; k<Nh; k++) {
      y[n] += h[k] * xt[n-k+Nh-1]; 
    }
  }
```

Note que el ciclo for más interno usa una instrucción del tipo 

``A = A + V1[ k1 ] * V2[ k2 ]``

que se lee como la multiplicación-acumulación de los elementos de dos vectores diferentes V1 y V2.

Esta instrucción, abreviada *MAC*, aparece también en filtros digitales IIR y en la implementación de algoritmos como la FFT, por lo que es fundamental en el procesamiento digital de señales. Por esta razón las operaciones MAC aparecen en las extensiones SIMD de muchos microprocesadores o como bloques hardware en FPGAs de alta gama.

![MAC-Simple y MAC SIMD](../img/apd_loopunrolling1.png)

En el figura de la izquierda se muestra a nivel funcional el esquema de una operación MAC. Asumiendo que el vector V1 y V2 son cantidades de 16 bits, el registro acumulador de salida A será de 32 bits.
 
Los microprocesadores modernos son en su mayoría de 64bits y aquellos que incluyen instrucciones SIMD, disponen de registros de 128bits o más. Si los datos de entrada de los vectores V1 y V2 son de 16bits, en un registro de 128 bits, podríamos alojar sin problema y de manera simultánea 128/16 = 8 elementos de los vectores V1 y V2, y hasta 128/32 = 4 registros acumuladores en dichos registros. Ese permite que a nivel de hardware podamos realizar 4 MACs en paralelo como se muestra en la figura de la derecha. Este hardware permite que un microprocesador ejecute una “sola” instrucción MAC múltiple que calcula 4 MACs simultáneamente en muy pocos ciclos de reloj.

A nivel de hardware, en microprocesadores con soporte SIMD contamos con la suficiente capacidad de cómputo para acelerar nuestros procesos de cálculo, por lo que para aprechar estas arquitecturas es necesario emplear una técnica que paralelize el algoritmo conocida como **_loop unrolling_**. Para ilustrarlo en la implementación de un filtro FIR, debemos recordar que la ecuación matemática que lo implementa:

$$y[n] = \sum_{k=0}^{Nh-1} h[k]x[n-k]$$

se puede expandir como se muestra a continuación

![FIR-SIMD](../img/apd_loopunrolling2.png)

Note que al expandir la sumatoria resultan varios términos MAC que se pueden calcular en paralelo, ya que no existe dependencia entre ellos. Por ejemplo, la primera fila (amarillo) de la ecuación anterior se puede implementar con un hardware como el mostrado en la figura de arriba para la MAC SIMD, la segunda fila (azul), sería otra MAC SIMD, y la tercera (verde), otra MAC SIMD, y así sucesivamente.

_Loop-unrolling_ solamente funciona si conocemos de antemano el número de veces que se repite un ciclo for, si este número número es múltiplo del número de unidades MAC disponibles en paralelo y si no existe independencia entre las iteraciones. Si alguna de las condiciones anteriores no se cumple, es imposible usar _loop-unrolling_. Afortunadamente en el algoritmo de un filtro FIR se sabe de antemano su longitud, y se puede diseñar para que Nh sea un número múltiplo de 2, 4 u 8.

En los compiladores de algunos microprocesadores existen mecanismos para hacer un llamado a instrucciones SIMD, conocido como ``intrinsics``. Esta técnica es algo compleja y no está disponible en todos los compiladores. En compiladores compatibles con GCC, se puede activar esta opción usando directivas ``#pragma`` para indicarle al compilador que tenemos un ciclo que se puede aplicar esta técnica y ayudarle al compilador a decidir cómo hacerlo.

Por ejemplo, le podemos decir al compilador que intente compilar el ciclo for original usando una estructura de _loop-unrolling_ donde el número de veces es múltiplo de 4 como se indica a continuación:	

```C 
  #pragma GCC unroll 4
  int32_t yt1 = 0;
  for(n=0;n<N_DATA;n++){
    yt1 += (int32_t)x[n] * (int32_t)x2[n]; 
  }
```

Aunque esto mejora el desempeño, es mejor reescribir el código para hacer explícito el _loop-unrolling_. En el caso anterior, una forma de hacerlo es reescribir el código haciendo explícitas las 4 instrucciones MAC como se muestra a continuación:

```C
  y1 = 0;
  y2 = 0;
  y3 = 0;
  y4 = 0;
  #pragma GCC unroll 4
  for(n=0;n<N_DATA;n+=4){
    y1 += x[n] * x2[n]; 
    y2 += x[n+1] * x2[n+1]; 
    y3 += x[n+2] * x2[n+2]; 
    y4 += x[n+3] * x2[n+3];
  }
  yt = y1+y2+y3+y4;	
```

Note que en este caso, se deben expandir los términos dentro del ciclo for haciendo explícito el offset en el acceso a cada arreglo, usar 4 acumuladores, y aumentar el índice n de 4 en 4, pues el ciclo en esta notación se debe repetir ``N_DATA/4`` veces.